### Install (if needed)
```bash
pip install -U langgraph
```


In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
try:
    from langgraph.checkpoint import MemorySaver
    checkpointer = MemorySaver()
except Exception:
    checkpointer = None

class AgentState(TypedDict, total=False):
    question: str
    context: str
    answer: str

def retrieve_node(state: AgentState) -> AgentState:
    import json as _json
    raw = search_policies(state.get("question", ""))
    data = _json.loads(raw)
    snippets = [r.get("snippet","") for r in data.get("results", [])]
    return {"context": "\n\n".join(snippets) if snippets else "No policy hits."}

def answer_node(state: AgentState) -> AgentState:
    q = state.get("question",""); cx = state.get("context","")
    prompt = ("Use ONLY the policy snippets to answer. If insufficient, say so.\n\n"
              f"Policy snippets:\n{cx}\n\nQuestion: {q}\n\nAnswer:")
    out = llm.invoke(prompt)
    text = getattr(out, "content", str(out))
    return {"answer": text}

graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve_node)
graph.add_node("answer", answer_node)
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "answer")
graph.add_edge("answer", END)
app = graph.compile(checkpointer=checkpointer) if checkpointer else graph.compile()

print(app.invoke({"question": "Does FleetAssist cover belongings in a rental car?"}).get("answer"))


NameError: name 'search_policies' is not defined

In [ ]:
# Optional: add a simple router for calc-like questions
import re

def route(state: AgentState) -> str:
    q = state.get("question","")
    return "calc" if re.search(r"\bCHF\b|\d+\s*[+\-*/]", q) else "retrieve"

def calc_node(state: AgentState) -> AgentState:
    q = state.get("question","")
    m = re.search(r"(\d+[\d\s+\-*/().]*)", q)
    expr = m.group(1) if m else None
    out = calculator(expr) if expr else "Error: no expression found"
    return {"context": f"Calculated: {expr} = {out}", "answer": str(out)}

g2 = StateGraph(AgentState)
g2.add_node("retrieve", retrieve_node)
g2.add_node("calc", calc_node)
g2.add_node("answer", answer_node)
g2.add_conditional_edges(START, route, {"retrieve": "retrieve", "calc": "calc"})
g2.add_edge("retrieve", "answer")
g2.add_edge("calc", "answer")
g2.add_edge("answer", END)
app_routed = g2.compile()

print(app_routed.invoke({"question": "If a laptop worth CHF 1,800 is damaged twice, what's the total?"}).get("answer"))


## Closing
Agents are **tools-first** systems. Ship‑ready means:
- Versioned graphs
- Safe MCP/allow‑listed tools
- Eval + observability

**Thanks!**